# Coleta de dados abertos da Câmara dos Deputados

**Objetivo**: Obter informações atualizadas sobre a(o)s deputada(o)s federais eleita(o)s e suas respectivas despesas realizadas, por meio dos recursos públicos destinados à chamada *cota parlamentar* - verba recebida pela(o)s deputada(o)s para custear o exercício de suas funções.  

**Fonte de dados**: <a href="https://dadosabertos.camara.leg.br/swagger/api.html">Dados Abertos da Câmara dos Deputados</a>

##Carga de pacotes

In [ ]:
# Carregamento dos pacotes necessários
import time
import json
import requests
import joblib
import os
from datetime import date
from google.colab import drive
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

##Credenciais de acesso

In [ ]:
# Autorizar acesso ao Google Drive
auth.authenticate_user()
drive_service = build('drive', 'v3')

In [ ]:
# Montar drive para armazenar conteúdo
drive.mount('/content/drive')

In [ ]:
# Criar pasta "DataSetCD" no Google Drive para salvar dados
os.mkdir("drive/MyDrive/DataSetCD")

## Extração de dados da fonte

In [ ]:
# Captura dos parlamentares em exercício atualmente - 57ª legislatura (2023-2027)
%%time
resp1 = requests.get("https://dadosabertos.camara.leg.br/api/v2/deputados?idLegislatura=57")
list_dept = resp1.json()['dados']
list_id = []
[list_id.append(x) for x in list({e['id'] for e in list_dept}) if x not in list_id] # Elimina eventuais repetições na lista de identificadores dos deputados
joblib.dump(list_id,f'drive/MyDrive/DataSetCD/listid') # Salvamento da lista de identificadores dos deputados na pasta DataSetCD
joblib.dump(list_dept,f'drive/MyDrive/DataSetCD/listdept') # Salvamento da lista de informações dos deputados na pasta DataSetCD
print(f'Número atual de deputados ativos: {len(list_id)}')

In [ ]:
# Carga de eventuais dados pré-existentes coletados das despesas
dic_dept_desp = {}
list_id = joblib.load(f'drive/MyDrive/DataSetCD/listid') # Carga dos identificadores dos deputados previamente salvos
for nid in list_id:
  try:
    listA = joblib.load(f'drive/MyDrive/DataSetCD/listdesp{nid}')
    dic_dept_desp.update({nid: listA})
  except:
    pass

In [ ]:
# Requisições das despesas dos deputados em exercício no(s) ano(s) e/ou no(s) mês(es) desejado(s)
%%time
for i in range(max(1, date.today().month-3), date.today().month): # Define intervalo para requisições de dados dos últimos 3 meses do ano corrente
  for nid in list_id: # Controla execução parcial das requisições
    listA = []
    for pg in range(1,100): # Define intervalo para requisições por página no ano corrente
      resp2 = requests.get(f"https://dadosabertos.camara.leg.br/api/v2/deputados/{str(nid)}/despesas?idLegislatura=57&ano={date.today().year}&mes={i}&pagina={pg}&itens=100")
      try:
        list0 = listA.copy()
        listA += resp2.json()['dados']
        if listA == list0:
          break
      except:
        pass
      time.sleep(0.5)
    try:
      dic_dept_desp[nid] += listA
      listB = []
      [listB.append(x) for x in dic_dept_desp[nid] if x not in listB] # Elimina eventuais repetições na lista atualizada das despesas de cada deputado
      dic_dept_desp[nid] = listB
    except:
      dic_dept_desp.update({nid: listA})
    joblib.dump(dic_dept_desp[nid],f'drive/MyDrive/DataSetCD/listdesp{nid}') # Salvamento parcial das despesas na pasta DataSetCD

In [ ]:
# Carga dos dados atualizados das despesas
dic1_dept_desp = {}
list_id = joblib.load(f'drive/MyDrive/DataSetCD/listid')
for nid in list_id:
  try:
    listA = joblib.load(f'drive/MyDrive/DataSetCD/listdesp{nid}')
    dic1_dept_desp.update({nid: listA})
  except:
    dic1_dept_desp.update({nid: []})

## Limpeza e transformação dos dados extraídos

In [ ]:
# Padronização dos e-mails dos deputados
list_dept = joblib.load(f'drive/MyDrive/DataSetCD/listdept') # Carga dos dados dos deputados previamente salvos
for i, elem in enumerate(list_dept):
    if elem['email'] is None:
        list_dept[i]['email'] = 'indisponível'

In [ ]:
# Limpeza dos dados atualizados: remover caracteres indesejados e nomes de fornecedores redundantes
for nid in list_id:
    for i, elem in enumerate(dic1_dept_desp[nid]):
        for char in "'.":
            dic1_dept_desp[nid][i]['nomeFornecedor'] = elem['nomeFornecedor'].replace(char,"")
            dic1_dept_desp[nid][i]['tipoDespesa'] = elem['tipoDespesa'].replace(char,"")
        if elem['nomeFornecedor'].startswith('Cia Aérea'):
            dic1_dept_desp[nid][i]['nomeFornecedor'] = elem['nomeFornecedor'].split('-')[1].strip()
        j = i
        while j >= 0:
            if dic1_dept_desp[nid][j]['cnpjCpfFornecedor']==dic1_dept_desp[nid][i]['cnpjCpfFornecedor'] and dic1_dept_desp[nid][j]['nomeFornecedor']!=dic1_dept_desp[nid][i]['nomeFornecedor']:
                dic1_dept_desp[nid][i]['nomeFornecedor'] = dic1_dept_desp[nid][j]['nomeFornecedor']
            j -= 1

In [ ]:
# Cópia dos dados limpos
dic2_dept_desp = dic1_dept_desp.copy()

In [ ]:
# Transformação dos dados - parte 1: explicitar mês das despesas
dic_mes = {1:'Jan', 2:'Fev', 3:'Mar', 4:'Abr', 5:'Mai', 6:'Jun', 7:'Jul', 8:'Ago', 9:'Set', 10:'Out', 11:'Nov', 12:'Dez'}
for nid in list_id:
    for i, elem in enumerate(dic2_dept_desp[nid]):
        dic2_dept_desp[nid][i]['mes'] = dic_mes[elem['mes']]

In [ ]:
# Transformação dos dados - parte 2: agrupar e simplificar os tipos de despesas, com base nas informações de https://www2.camara.leg.br/comunicacao/assessoria-de-imprensa/guia-para-jornalistas/cota-parlamentar
for nid in list_id:
    for i, e in enumerate(dic2_dept_desp[nid]):
        if e['tipoDespesa'] == 'AQUISIÇÃO DE TOKENS E CERTIFICADOS DIGITAIS':
            dic2_dept_desp[nid][i]['tipoDespesa'] = 'TOKENS'
        elif e['tipoDespesa'] == 'ASSINATURA DE PUBLICAÇÕES':
            dic2_dept_desp[nid][i]['tipoDespesa'] = 'PUBLICAÇÕES'
        elif e['tipoDespesa'] == 'FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR':
            dic2_dept_desp[nid][i]['tipoDespesa'] = 'ALIMENTAÇÃO'
        elif e['tipoDespesa'] == 'PARTICIPAÇÃO EM CURSO, PALESTRA OU EVENTO SIMILAR':
            dic2_dept_desp[nid][i]['tipoDespesa'] = 'EVENTOS INSTRUCIONAIS'
        elif e['tipoDespesa'].startswith('LOCAÇÃO'):
            dic2_dept_desp[nid][i]['tipoDespesa'] = 'LOCAÇÃO DE VEÍCULOS'
        elif e['tipoDespesa'].startswith('PASSAGE'):
            dic2_dept_desp[nid][i]['tipoDespesa'] = 'VIAGENS'
        elif e['tipoDespesa'].startswith('SERVIÇO') or e['tipoDespesa'] == 'TELEFONIA':
            dic2_dept_desp[nid][i]['tipoDespesa'] = 'SERVIÇOS'
        else:
            dic2_dept_desp[nid][i]['tipoDespesa'] = e['tipoDespesa'].split()[0].replace(',','')

## Exportação dos dados pré-processados

In [ ]:
# Exportação de dados selecionados dos deputados para arquivo JSON
list_dept_mod = []
for elem in list_dept:
    list_dept_mod.append({"id": elem['id'], "nome": elem['nome'], "email": elem['email'],\
                            "partido": elem['siglaPartido'], "uf": elem['siglaUf'], "foto": elem['urlFoto']})
with open(f'deputadosLEG57_{date.today().year}.json', 'w', encoding='utf8') as arq:
    json.dump(list_dept_mod, arq, ensure_ascii=False)

In [ ]:
# Upload das informações do arquivo JSON dos deputados para o Google Drive
file_metadata1 = {
  'name': f'deputadosLEG57_{date.today().year}',
  'mimeType': 'application/json'
}
media1 = MediaFileUpload(f'deputadosLEG57_{date.today().year}.json',
                        mimetype='application/json',
                        resumable=True)
file_list = drive_service.files().list().execute()['files']
if any(file['name'] == f'deputadosLEG57_{date.today().year}' for file in file_list):
  fileId1 = [file['id'] for file in file_list if file['name']==f'deputadosLEG57_{date.today().year}'][0]
  updated1 = drive_service.files().update(fileId=fileId1,
                                        body=file_metadata1,
                                        media_body=media1).execute()
else:
  created1 = drive_service.files().create(body=file_metadata1,
                                       media_body=media1,
                                       fields='id').execute()

In [ ]:
# Exportação de dados selecionados das despesas para arquivo JSON
list_dept_desp = []
for chv in dic2_dept_desp.keys():
    for elem in dic2_dept_desp[chv]:
        list_dept_desp.append({"id": chv, "tipo": elem['tipoDespesa'], "iddoc": elem['codDocumento'],\
                                "ano": elem['ano'], "mes": elem['mes'], "data": elem['dataDocumento'],\
                                "nomefornecedor": elem['nomeFornecedor'], "codfornecedor": elem['cnpjCpfFornecedor'],\
                                "valor": elem['valorLiquido']})
with open(f'despesasLEG57_{date.today().year}.json', 'w', encoding='utf8') as arq:
    json.dump(list_dept_desp, arq, ensure_ascii=False)

In [ ]:
# Upload das informações do arquivo JSON das despesas para o Google Drive
file_metadata2 = {
  'name': f'despesasLEG57_{date.today().year}',
  'mimeType': 'application/json'
}
media2 = MediaFileUpload(f'despesasLEG57_{date.today().year}.json',
                        mimetype='application/json',
                        resumable=True)
file_list = drive_service.files().list().execute()['files']
if any(file['name'] == f'despesasLEG57_{date.today().year}' for file in file_list):
  fileId2 = [file['id'] for file in file_list if file['name']==f'despesasLEG57_{date.today().year}'][0]
  updated2 = drive_service.files().update(fileId=fileId2,
                                        body=file_metadata2,
                                        media_body=media2).execute()
else:
  created2 = drive_service.files().create(body=file_metadata2,
                                       media_body=media2,
                                       fields='id').execute()